In [1]:
spam_text= ['Send us your password', 'review us', 'Send your password', 'Send us your account']

ham_text= ['Send us your review', 'review your password']

spam = {}
for i in spam_text:
    for j in i.lower().split(' '):
        if j not in spam:
            spam[j] = 1
        else:
            spam[j] += 1

print("Spam Dictionary:")            
print(spam)
print("\n")

ham = {}
for i in ham_text:
    for j in i.lower().split(' '):
        if j not in ham:
            ham[j] = 1
        else:
            ham[j] += 1

print("Ham Dictionary:")
print(ham)

Spam Dictionary:
{'send': 3, 'us': 3, 'your': 3, 'password': 2, 'review': 1, 'account': 1}


Ham Dictionary:
{'send': 1, 'us': 1, 'your': 2, 'review': 2, 'password': 1}


In [2]:
#2/13
p_password_given_spam = spam['password']/sum(spam.values())
print(p_password_given_spam)

0.15384615384615385


In [3]:
#1/7
p_password_given_ham = ham['password']/sum(ham.values())
print(p_password_given_ham)

0.14285714285714285


In [4]:
p_spam = 4/6
p_ham = 2/6

Bayes' Rule: $P(spam \mid password) = (P(password \mid spam) P(spam))/ P(password)$

LOTP: $P(password) = P(password \mid spam) P(spam) + P(password \mid ham) P(ham)$


In [5]:
p_password = (p_password_given_spam * p_spam) + (p_password_given_ham * p_ham)

In [6]:
(p_password_given_spam *p_spam) / p_password

0.6829268292682927

The Bayes Theorem : $P(spam | w_1, w_2, ..., w_n) = {P(w_1, w_2, ..., w_n | spam)P(spam)}/{P(w_1, w_2, ..., w_n)}$

In [14]:
import os
import re
import string
import math

In [47]:
class SpamDetector(object):
    """Implementation of Naive Bayes for binary classification"""
    def clean(self, s):
        translator = str.maketrans("", "", string.punctuation)
        return s.translate(translator)
 
    def tokenize(self, text):
        text = self.clean(text).lower()
        return re.split("\W+", text)
 
    def get_word_counts(self, words):
        word_counts = {}
        for word in words:
            word_counts[word] = word_counts.get(word, 0.0) + 1.0
        return word_counts
    
    def fit(self, X, Y):
        self.num_messages = {}
        self.log_class_priors = {}
        self.word_counts = {}
        self.vocab = set()

        n = len(X)
        self.num_messages['spam'] = sum(1 for label in Y if label == 1)
        self.num_messages['ham'] = sum(1 for label in Y if label == 0)
        self.log_class_priors['spam'] = math.log(self.num_messages['spam'] / n)
        self.log_class_priors['ham'] = math.log(self.num_messages['ham'] / n)
        self.word_counts['spam'] = {}
        self.word_counts['ham'] = {}

        for x, y in zip(X, Y):
            c = 'spam' if y == 1 else 'ham'
            counts = self.get_word_counts(self.tokenize(x))
            for word, count in counts.items():
                if word not in self.vocab:
                    self.vocab.add(word)
                if word not in self.word_counts[c]:
                    self.word_counts[c][word] = 0.0

                self.word_counts[c][word] += count
                
    def predict(self, X):
        result = []
        for x in X:
            counts = self.get_word_counts(self.tokenize(x))
            spam_score = 0
            ham_score = 0
            for word, _ in counts.items():
                if word not in self.vocab: continue

                # add Laplace smoothing
                log_w_given_spam = math.log( (self.word_counts['spam'].get(word, 0.0) + 1) / (self.num_messages['spam'] + len(self.vocab)) )
                log_w_given_ham = math.log( (self.word_counts['ham'].get(word, 0.0) + 1) / (self.num_messages['ham'] + len(self.vocab)) )

                spam_score += log_w_given_spam
                ham_score += log_w_given_ham

            spam_score += self.log_class_priors['spam']
            ham_score += self.log_class_priors['ham']

            if spam_score > ham_score:
                result.append(1)
            else:
                result.append(0)
        return result

In [49]:
if __name__ == '__main__':
    from sklearn.model_selection import train_test_split

    # import/clean/label your data
    data = pd.read_csv('Notebooks/Datasets/spam.csv',encoding='latin-1')
    data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
    data = data.rename(columns={"v1":'label', "v2":'text'})
    print(data.head())
    tags = data["label"]
    texts = data["text"]

    # create texts and tags
    X, y = texts, tags
    print(len(X))
    
    # transform text into numerical vectors
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    # instantiate your SpamDetector
    MNB = SpamDetector()
    # fit to model, with the trained part of the dataset
    MNB.fit(X_train.values, y_train.values)
    print(MNB.num_messages)
#     print(MNB.word_counts)
    # make predictions
    pred = MNB.predict(X_test.values)
    true = y_test.values
    # test for accuracy
    accuracy = sum(1 for i in range(len(pred)) if pred[i] == true[i]) / float(len(pred))
    print("{0:.4f}".format(accuracy))

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
5572


TypeError: fit() missing 1 required positional argument: 'Y'

In [50]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# TODO: Prepare the dataset
data = pd.read_csv('Notebooks/Datasets/spam.csv',encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

# TODO: create texts and tags
tags = data["label"]
texts = data["text"]
X, y = texts, tags


# TODO: split the data into train vs test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


# TODO: transform text into numerical vectors
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

# instantiate Multinomial Naive Bayes model
nb = MultinomialNB()
# fit to model, with the trained part of the dataset
nb.fit(X_train_vec, y_train)
X_test_vec = vectorizer.transform(X_test)
# make prediction
y_pred_class = nb.predict(X_test_vec)
# test accurarcy of prediction
metrics.accuracy_score(y_test, y_pred_class)

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


0.9856424982053122

In [52]:
metrics.f1_score(y_test, y_pred_class, pos_label='spam')

0.9438202247191012